# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계1 : 응급상황 음성 인식 및 요약**

## **0.미션**

단계 1에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### (1) 미션1
* 음성인식 : STT(Speech-to-Text)
    * 사용 모델 : OpenAI의 **Whisper-1**
    * 제공받은 음성 파일과 새로 제작하는 5건 이상의 음성파일을 텍스트로 변환하고, 변환작업이 잘 되는지 확인해 봅시다.

### (2) 미션2
* 텍스트 요약 및 핵심 키워드 도출
    * 사용 모델 : OpenAI의 **GPT-3.5-turbo**
    * 내용 요약과 주요 키워드를 도출하도록
    프롬프트 입력과 출력을 구성하고 테스트 해 봅시다.

* [추가]응급실 현황 다운로드(이 데이터는 단계3에서 필요합니다.)



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import openai
from openai import OpenAI
import json

# 더 필요한 라이브러리 추가 -------------




### (3) OpenAI API Key 환경 변수 설정

* 제공받은 open ai api key를 **api_key.txt** 파일에 저장합니다.
    * (제공받은 api_key.txt 파일은 비어 있습니다.)

* 다음 코드를 통해 환경변수로 등록 합니다.

In [ ]:
def load_file(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

# API 키 로드 및 환경변수 설정
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 결과는 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2. 미션1 : STT**

### (1) 제공된 데이터 변환
* 세부사항
    * 사용 모델 : whisper-1
    * 제공 받은 오디오 파일을 읽어서 텍스트로 변환시켜 봅시다.
        * 반복문을 통해 파일 하나씩 읽어서 텍스트 변환
        * 변환된 텍스트를 데이터 프레임에 추가

|filename|text|
|----|----|
|audio3.mp3|어쩌구 저쩌구...급해요.|

* 음성파일 변환

In [ ]:
# 음성파일 경로 지정
audio_path = path + 'audio/'
audio_path

'/content/drive/MyDrive/project6_2/audio/'

In [ ]:
# OpenAI 클라이언트 생성
client = OpenAI()

In [ ]:
# 위스퍼 모델 사용
filename = 'audio2.mp3'
audio_file = open(audio_path + filename, "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="text",
    temperature=0.0
)

print(transcript, type(transcript))

119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?
 <class 'str'>


* 음성파일 변환 함수 생성

In [ ]:
def audio_to_text(audio_path, filename):
    client = OpenAI()
    audio_file = open(audio_path + filename, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        language="ko",
        response_format="text",
        temperature=0.0
)
    return transcript

In [ ]:
# 음성파일 이름을 리스트에 담기
file_names = [f for f in os.listdir(audio_path) if os.path.isfile(os.path.join(audio_path, f))]
print(file_names)

['audio3.mp3', 'audio1.mp3', 'audio5.mp3', 'audio4.mp3', 'audio2.mp3']


In [ ]:
# 반복문을 통해, 파일 하나씩 읽어서 텍스트 변환, 변환된 텍스트를 데이터 프레임에 추가

# 빈 데이터프레임 선언
result = pd.DataFrame(columns=['filename', 'text'])

# 반복문 수행하면서 오디오 변환
for i, filename in enumerate(file_names):
    transcript = audio_to_text(audio_path, filename)
    # 데이터 프레임에 추가 : filename, text
    temp = pd.DataFrame([{'filename': filename, 'text': transcript}])
    result = pd.concat([result, temp])
    print(filename)

result.reset_index(drop=True, inplace=True)
result

audio3.mp3
audio1.mp3
audio5.mp3
audio4.mp3
audio2.mp3


,filename,text
0,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...
1,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...
2,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...
3,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리..."
4,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...


### (2) 오디오 데이터 추가 수집(제작) 및 변환

* 세부사항
    * 응급 상황에 맞는 음성 녹음하기
        * 응급 등급별 1개 이상씩(총 5개 이상)
    * 반복문을 통해 모든 음성 파일 데이터 변환 : STT
        * 변환 내용은 위에서 저장한 데이터프레임에 추가
    * 변환 후 음성 내용과 변환 결과를 비교


## **3. 미션2 : Summary**

* 세부사항
    * 문서요약 예제 파일을 참조하여 테스트 해 봅니다.
    * 코드를 참조하여, 원하는 형식에 맞게 요약이 되도록 프롬프트를 구성합니다.
        * 요약 시 중요 키워드들이 함께 도출되도록 합니다.
        * 가능하다면, 요약 문장 길이에 제한을 둡시다.
    * 반복문을 통해 요약하고, 결과를 데이터프레임에 추가합니다.
        * summary 열을 추가하고, 요약 결과를 입력
            * 요약결과와 키워드는 하나의 문자열로 붙여서 summary열에 추가

### (1) 문서 요약

* 문서 요약 예제

In [ ]:
input_text = '''
한국은행 총재가 "올해 성장률이 기존 전망치 2.4%보다 낮아질 가능성이 크다"며 "2.2∼2.3% 정도로 떨어지지 않을까 생각한다"고 밝혔습니다.
이 총재는 오늘(29일) 국회 기획재정위원회 국정감사에 출석해 한은의 전망을 크게 밑돈 3분기 성장률을 바탕으로 올해 성장률 전망치가 조정될 가능성에 대해 이렇게 말했습니다.
성장률 하락의 가장 큰 요인인 수출 감소의 배경에 대해 이 총재는 "금액 기준으로 봐서는 수출이 안 떨어졌는데, 수량을 기준으로 떨어졌다"며 "자동차 파업 등 일시적 요인과 화학제품·반도체의 중국과 경쟁 등으로 수량이 안 늘어나는 것 같은데, 원인을 더 분석해봐야 할 사안"이라고 진단했습니다.
다음 달 28일 열릴 기준금리 결정 방향에 대해서는 "금리 결정할 때 하나의 변수만 보지 않고 종합적으로 보는데, 우선 미국 대선과 연방준비제도 금리 결정으로 경제 상황이 어떻게 변할지 보겠다"고 밝혔습니다.
또 "아울러 이후 달러가 어떻게 될지, 수출 등 내년 경제 전망과 거시안전성 정책이 부동산·가계부채에 미치는 영향 등도 고려해 결정하겠다"고 말했습니다.
'''

system_role = '''당신은 신문기사에서 핵심을 요약하는 어시스턴트입니다.
응답은 다음의 형식을 지켜주세요
{"summary": \"텍스트 요약\",
"keyword" : \"핵심 키워드(3가지)\"}
'''

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": system_role
        },
        {
            "role": "user",
            "content": input_text
        }
    ]
)

answer = response.choices[0].message.content
print(answer)

{'summary': "한국은행 총재는 올해 성장률이 2.4%보다 낮아질 가능성이 있으며, 2.2∼2.3%로 예상된다고 밝혔다. 성장률 하락의 요인으로는 수출 감소와 관련된 일시적 요인과 중국과의 경쟁이 있는 것으로 보고 있다. 기준금리 결정 방향은 미국 대선, 연방준비제도의 결정 등 다양한 변수를 고려하여 결정할 것이라고 밝혔다.", 
'keyword': "한국은행, 총재, 성장률"}


* 문서 요약 함수로 생성

In [ ]:
def text_summary(input_text):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    system_role = '''당신은 응급상황에 대한 텍스트에서 핵심 내용을 훌륭하게 요약해주는 어시스턴트입니다.
    응답은 다음의 형식을 지켜주세요.
    {"summary": \"텍스트 요약\",
    "keyword" : \"핵심 키워드(3가지)\"}
    '''
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )
    answer = response.choices[0].message.content
    parsed_answer = json.loads(answer)

    summary = parsed_answer["summary"]
    keyword = parsed_answer["keyword"]

    return summary + ', ' + keyword


* 저장된 text를 하나씩 불러와서 요약하고 다시 저장하기

In [ ]:
# result에 summary 열 추가
result['summary'] = ''
result

,filename,text,summary
0,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...,
1,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...,
2,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...,
3,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리...",
4,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...,


In [ ]:
for i, t in enumerate(result['text']) :
    input_text = t
    summary = text_summary(input_text)

    result.loc[i, 'summary'] = summary

In [ ]:
# 저장 결과 확인하기
result

,filename,text,summary
0,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...,"동생이 코에 콩이 들어가서 한쪽 코가 막혔는데, 빼도 안 빠져서 고통스러워하고 있음..."
1,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...,남성 환자가 계단에서 넘어져 머리를 다쳐 출혈하고 있으며 어지러운 증상이 나타나고 ...
2,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...,화장실에서 미끄러져 엉덩방아를 찍었고 꼬리뼈가 계속 아파서 점점 아픔이 심해지는 상...
3,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리...",머리를 박아서 3시간이 지난 상태에서 머리가 어지럽고 속이 매스꺼운 증상이 나타납니...
4,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...,응급상황 발생 시 119에 전화한 후 38도 이상의 열이 있고 머리가 아프며 어지러...


### (2) 전국 병원 응급실 정보 수집



#### 1) 인증키 발급

* 인증키 발급 절차
    * 1) data.go.kr 회원가입
    * 2) 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
https://www.data.go.kr/data/15000563/openapi.do 로 이동
    * 3) 활용신청
        * 활용목적 : 기타(개인 학습 용도)
        * 상세 기능선택
            * 응급의료기관 목록정보 조회
            * 응급의료기관 위치정보 조회
            * 응급의료기관 기본정보 조회
    * 4) 인증키 확인
        * 마이페이지 > Open API > 활용신청현황
        * [승인] 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
        * 일반 인증키(Decoding) 이용

#### 2) 데이터 수집

In [ ]:
# path 확인
path

'/content/drive/MyDrive/project6_2/'

In [ ]:
# 응급실 데이터 수집하기

url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire'
serviceKey = 'iPNPiMT14DMTyAInj8Infw+C4s0a8RuazywUYiQf4m6fT7I1clyKLDaZp8XkGs1drKu2ID1vX6gMDrByYFPGXg=='  # 여러분의 일반 인증키(Decoding)

params = {
    'serviceKey': serviceKey,
    'pageNo': '1', 'numOfRows': '1000',  # 전체 응급실 수가 500여개 됨. 1000개면 충분
    'format': 'xml'
}

response = requests.get(url, params=params)

# 정상 수행 되었다면 200
print(response)

<Response [200]>


In [ ]:
# response xml에서 주요 정보 찾기
root = ET.fromstring(response.text)

data = []

for item in root. findall('.//item'):
    duty_name = item.findtext('dutyName')
    duty_addr = item.findtext('dutyAddr')
    duty_tel1 = item.findtext('dutyTel1')
    duty_tel3 = item.findtext('dutyTel3')
    duty_emcls_name = item.findtext('dutyEmclsName')
    lat = item.findtext('wgs84Lat')
    lon = item.findtext('wgs84Lon')

    data.append({
        "병원이름": duty_name,
        "주소": duty_addr,
        "응급의료기관 종류": duty_emcls_name,
        "전화번호 1": duty_tel1,
        "전화번호 3": duty_tel3,
        "위도": lat,
        "경도": lon
        })

# 데이터프레임으로 변환
em_df = pd.DataFrame(data)

In [ ]:
# csv 파일로 저장
em_df.to_csv(path + '응급실정보.csv', index=False)

## **Mission Complete!**

수고 많았습니다!